In [1]:
import gc
import glob
import os
import re

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, StratifiedKFold

%matplotlib inline

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

'wget' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [4]:
!cat aclImdb/README

'cat' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [2]:
from sklearn.datasets import load_files
train_review = load_files('./aclImdb/train/', encoding='utf-8')

In [3]:
x_train, y_train = train_review.data, train_review.target
test_review = load_files('./aclImdb/test/', encoding='utf-8')
x_test, y_test = test_review.data, test_review.target
# ラベルの0,1と意味の対応の表示
print(train_review.target_names)

['neg', 'pos']


In [88]:
print("x : {}".format(y_train[0]))

x : 1


In [10]:
mini_dataset = \
  ["This movie is very good.",
  "This film is a good",
  "Very bad. Very, very bad."]

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
bow = (vectorizer.fit_transform(mini_dataset)).toarray()
# DataFrameにまとめる
df = pd.DataFrame(bow, columns=vectorizer.get_feature_names())
display(df)

,a,bad,film,good,is,movie,this,very
0,0,0,0,1,1,1,1,1
1,1,0,1,1,1,0,1,0
2,0,2,0,0,0,0,0,3


テキストはBoWにより各サンプルが語彙数の次元を持つ特徴量となり、機械学習モデルへ入力できるようになります。  
この時使用したテキスト全体のことを コーパス と呼びます。  
語彙はコーパスに含まれる言葉よって決まり、それを特徴量としてモデルの学習を行います。  
そのため、テストデータではじめて登場する語彙はベクトル化される際に無視されます。

BoWは厳密には単語を数えているのではなく、 トークン（token） として定めた固まりを数えます。  
何をトークンとするかはCountVectorizerでは引数token_patternで 正規表現 の記法により指定されます。  
デフォルトはr'(?u)\b\w\w+\b'ですが、上の例ではr'(?u)\b\w+\b'としています。  
デフォルトでは空白・句読点・スラッシュなどに囲まれた2文字以上の文字を1つのトークンとして抜き出すようになっているため、
「a」や「I」などがカウントされません。  
英語では1文字の単語は文章の特徴をあまり表さないため、除外されることもあります。  
しかし、上の例では1文字の単語もトークンとして抜き出すように引数を指定しています。  

In [12]:
# ngram_rangeで利用するn-gramの範囲を指定する
vectorizer = CountVectorizer(ngram_range=(2, 2), token_pattern=r'(?u)\b\w+\b')
bow_train = (vectorizer.fit_transform(mini_dataset)).toarray()
df = pd.DataFrame(bow_train, columns=vectorizer.get_feature_names())
display(df)

,a good,bad very,film is,is a,is very,movie is,this film,this movie,very bad,very good,very very
0,0,0,0,0,1,1,0,1,0,1,0
1,1,0,1,1,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,2,0,1


# 【問題１】 BoWのスクラッチ実装

In [3]:
bow_dataset = \
["This movie is SOOOO funny!!!",
"What a movie! I never",
"best movie ever!!!!! this movie"]

In [80]:
# column list を作成する。
# index に合わせて、カウントしていく。

get_columns = []
for str1 in bow_dataset:
    temp = str1.replace("!","")
    temp = temp.split(" ")
    for wrd in temp:
        if wrd in get_columns:
            pass
        else:
            get_columns.append(wrd)

df = pd.DataFrame(index=range(len(bow_dataset)),columns=df_columns)

for i,str1 in enumerate(bow_dataset):
    temp = str1.replace("!","")
    temp = temp.split(" ")
    df.iloc[i] = 0
    for wrd in temp:
        df.iloc[i][wrd]  += 1


In [81]:
df

,This,movie,is,SOOOO,funny,What,a,I,never,best,ever,this
0,1,1,1,1,1,0,0,0,0,0,0,0
1,0,1,0,0,0,1,1,1,1,0,0,0
2,0,2,0,0,0,0,0,0,0,1,1,1


## 【問題2】TF-IDFの計算

In [5]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hirot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
stopWords = stopwords.words('english')

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = x_train
vectorizer = TfidfVectorizer(stop_words=stopWords,token_pattern=r'(?u)\b\w+\b', max_features=50)

# norm = Noneといれることで正則化が行われなくなる。

# fit することでvocabluaryが設定される。
X=vectorizer.fit_transform(corpus)
#vocab = vectorizer.vocabulary_

#print(vectorizer.get_feature_names())

#X = vectorizer.fit_transform(corpus).toarray()
# スパース表現→配列表現にできる。

In [8]:
X[1].toarray()

array([[0.        , 0.12785745, 0.39821412, 0.        , 0.43800216,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.22692136, 0.        , 0.        , 0.        , 0.12816108,
        0.12526271, 0.10609775, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.130662  ,
        0.        , 0.42312832, 0.32387604, 0.13799925, 0.2456185 ,
        0.        , 0.17039113, 0.        , 0.14138692, 0.23765553,
        0.        , 0.        , 0.        , 0.11557921, 0.        ,
        0.16340563, 0.        , 0.        , 0.        , 0.11146519,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

## 【問題3】TF-IDFを用いた学習

In [18]:
## x_train（corpus）とx_testでインスタンスを分ける。この際、x_trainのインスタンスで決定されたボキャブラリをx_testに流用する。
## これによりaccuracyが向上する。

vectorizer1 = TfidfVectorizer(stop_words=stopWords,token_pattern=r'(?u)\b\w+\b', max_features=5000)
X=vectorizer1.fit_transform(corpus)
vocab = vectorizer1.vocabulary_

vectorizer2 = TfidfVectorizer(stop_words=stopWords,token_pattern=r'(?u)\b\w+\b', max_features=5000,vocabulary=vocab)
X_test = vectorizer2.fit_transform(x_test)


In [19]:
X_test.shape

(25000, 5000)

In [20]:
from sklearn import linear_model
Y = y_train
clf = linear_model.SGDClassifier()
clf.fit(X, Y)
y_pred = clf.predict(X_test)

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.88192